In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import time
import datetime as dt
from sklearn import preprocessing ,model_selection,metrics,feature_selection
from sklearn.model_selection import GridSearchCV , learning_curve
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn import neighbors,linear_model,svm,tree,ensemble
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import silhouette_samples,silhouette_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, plot_roc_curve



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("../input/ecommerce-data/data.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df.info()

In [ ]:
df.dropna(axis=0,inplace=True)
df.shape

In [ ]:
x_values = df['Quantity']
sns.kdeplot(x_values,color='r',shade=True,label='Quantity')
plt.xlabel('Quantity')
plt.ylabel('Probability Density')
plt.show()

In [ ]:
x_values =df['UnitPrice']
sns.kdeplot(x_values,color='k',shade=True,label='UnitPrice')
plt.xlabel('')

In [ ]:
df = df[df['Quantity']>=0]
df = df[df['UnitPrice']>=0]
print(df.shape)

In [ ]:
df.info()
print(df.describe())
df.head()

In [ ]:
df['TotalQuantity'] = df['Quantity']*df['UnitPrice']
df.head()

In [ ]:
print(df[['InvoiceNo','Country']].groupby('Country').count().sort_values('InvoiceNo',ascending=False))


In [ ]:
country = ['Netherlands','United Kingdom','Belgium','Germany'   , 'France' , 'EIRE'  , 'Spain'    , 'Portugal'  , 'Others','Switzerland']
invoice =[2363,354345,2031,9042,8342,7238,2485 ,1462,8774,1842]
fig = plt.figure(figsize=(10,10))
plt.pie(invoice,labels=country)
plt.show()


In [ ]:
df['Country'].value_counts().head(15).plot(kind='bar')


In [ ]:
data_temp = df.groupby(['Country'])['TotalQuantity'].sum().reset_index().sort_values('TotalQuantity',ascending=False).head(7)
print(data_temp)
print(data_temp.plot(x='Country',y='TotalQuantity',kind='bar',color=['r','g','b','y','r','k','b']))

In [ ]:
df[df['TotalQuantity']==df['TotalQuantity'].max()]

<p>Conclusion: Since the given website is a United Kindom originated website. All the variables such as No. of Customers and the Gross total sales is dominated by the United Kingdom. The remaning portion is occupied by the neighbouring Europian Countries</p>

In [ ]:
items = df['Description'].value_counts().head(15)
print(items)

In [ ]:
item_counts = df['Description'].value_counts().sort_values(ascending=False).head(20)
sns.barplot(item_counts.index,item_counts.values,palette='Greens_r')
plt.ylabel('Counts')
plt.title('Which items were bought more often?')
plt.show()

<p>This analysis gave us a general idea of which product was most demanded by the customers and the bar plot is used for the same visulaization</p>

In [ ]:
print(df[['InvoiceNo','Country','TotalQuantity','CustomerID']].sort_values('TotalQuantity',ascending=False).head(15))


In [ ]:
df['Date'] = df['InvoiceDate'].apply(lambda x : x.date())
df.head()

In [ ]:
recency_df = df.groupby('CustomerID',as_index=False)['Date'].max()
recency_df.columns = ['CustomerID','LastPurchase']
recency_df.head(10)

In [ ]:
now = dt.date(2021,12,1)
print(now)

In [ ]:
recency_df['Recency'] = recency_df['LastPurchase'].apply(lambda x : (now-x).days)
recency_df.drop('LastPurchase',axis = 1,inplace=True)
recency_df.head(5)                                                          

In [ ]:
temp = df.copy()
temp.drop_duplicates(['InvoiceNo','CustomerID'],keep='first',inplace=True)
frequency_df = temp.groupby(by=['CustomerID'], as_index=False)['InvoiceNo'].count()
frequency_df.columns = ['CustomerID','Frequency']
frequency_df.head()

In [ ]:
monetary_df = df.groupby('CustomerID',as_index=False).agg({'TotalQuantity':'sum'})
monetary_df.columns = ['CustomerID','TotalQuantity']
monetary_df.head(10)

In [ ]:
rfm_table = recency_df.merge(frequency_df,on='CustomerID').merge(monetary_df,on='CustomerID')
rfm_table.set_index('CustomerID',inplace=True)
rfm_table.head(10)

In [ ]:
features = rfm_table.columns
rfm_table.shape

In [ ]:
sns.heatmap(rfm_table.corr(),cmap="YlGnBu",annot=True)

In [ ]:
sns.pairplot(rfm_table,diag_kind = 'hist')


In [ ]:
from sklearn.preprocessing import PowerTransformer
pt = PowerTransformer()
rfm_table = pd.DataFrame(pt.fit_transform(rfm_table))
rfm_table.columns = features
rfm_table.head()

In [ ]:
sc = StandardScaler()
rfm_scaled = sc.fit_transform(rfm_table)
rfm_scaled[:5]

In [ ]:
from sklearn.decomposition import PCA
pc = PCA()
pc_transformed = pc.fit_transform(rfm_scaled)


In [ ]:
pc.components_

In [ ]:
pc.explained_variance_

In [ ]:
var_exp = pc.explained_variance_ratio_
var_exp

In [ ]:
np.cumsum(var_exp)

In [ ]:
pc.mean_

In [ ]:
pc.n_features_

In [ ]:
plt.figure(figsize=(6,4))
plt.bar(range(3),var_exp,alpha=0.3,align='center',label='Individual explained variance')
plt.step(range(3),np.cumsum(var_exp),where='mid',label='Cumulative explained variance')
plt.ylabel('Explained Variance Ratio')
plt.xlabel('Principal Components')
plt.legend(loc='best')
plt.tight_layout()
plt.show()

In [ ]:
X = rfm_table.copy()
pc = PCA(n_components=2)
df_pca = pc.fit_transform(X)
df_pca = pd.DataFrame(df_pca)
df_pca.head()

In [ ]:
X = df_pca.copy()

In [ ]:

from sklearn.cluster import KMeans
cluster_range = range(1,5)
cluster_errors = []
cluster_sil_store = []
for num in cluster_range:
    clusters = KMeans(num, n_init = 100,init='k-means++',random_state=0)
    clusters.fit(X)
    labels = clusters.labels_                     # capture the cluster lables
    centroids = clusters.cluster_centers_         # capture the centroids
    cluster_errors.append( clusters.inertia_ )    # capture the intertia
clusters_df = pd.DataFrame({ "num_clusters":cluster_range, "cluster_errors": cluster_errors} )
clusters_df[0:10]

In [ ]:
plt.figure(figsize=(15,6))
plt.plot(clusters_df["num_clusters"],clusters_df["cluster_errors"],marker = 'o')
plt.xlabel('count of clusters')
plt.ylabel('error')

In [ ]:
for num in range(2,16):
    clusters = KMeans(n_clusters=num,random_state=0)
    labels = clusters.fit_predict(df_pca)
    
    sil_avg = silhouette_score(df_pca, labels)
    print('For',num,'The Silhouette Score is =',sil_avg)

In [ ]:
kmeans = KMeans(n_clusters = 4)
kmeans = kmeans.fit(df_pca)
labels = kmeans.predict(df_pca)
centroids = kmeans.cluster_centers_

print(labels)
print()
print('Cluster Centers')
print(centroids)

In [ ]:
df_pca['Clusters'] = labels
df_pca.head()

In [ ]:
df_pca['Clusters'].value_counts()

In [ ]:
df_pca.boxplot(by = 'Clusters',figsize=(15,6))
plt.show()

In [ ]:
sns.pairplot(df_pca,diag_kind='hist',hue='Clusters')

In [ ]:
df_pca.head(5)

In [ ]:
X = df_pca[[0,1]]
Y = df_pca['Clusters']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42, stratify=Y)
lr = LogisticRegression(max_iter=1000,random_state=0)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print('Test accuracy = ', accuracy_score(y_test, y_pred))

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 4) 
  
knn.fit(X_train, y_train) 
pred = knn.predict(X_test) 
  
# Predictions and Evaluations 
# Let's evaluate our KNN model !  
from sklearn.metrics import classification_report, confusion_matrix 
print(confusion_matrix(y_test, pred)) 
  
print(classification_report(y_test, pred)) 

In [ ]:
from sklearn.tree import DecisionTreeRegressor
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42, stratify=Y)
regressor = DecisionTreeRegressor(random_state = 213)
regressor.fit(X_train,y_train)

pred = regressor.predict(X_test) 
print(confusion_matrix(y_test, pred)) 
  
print(classification_report(y_test, pred)) 